# droitGPT

# LLM: Qwen-1_8B-Chat-Int4

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MAX_NEW_TOKENS = 8192
"""
Source: 
- context length: https://huggingface.co/Qwen/Qwen-1_8B-Chat-Int4
"""
MODEL_ID = "Qwen/Qwen-1_8B-Chat-Int4"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID,device_map="auto",trust_remote_code=True)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=MAX_NEW_TOKENS)
hf = HuggingFacePipeline(pipeline=pipe)

# Test LLM with specific context

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """
Vous êtes un assistant spécialisé en droit français.
<context>
{context}
</context>
Question: {input}
Réponse: Expliquez les termes simplement.
"""

prompt = PromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(hf, prompt)

In [ ]:
from langchain_core.documents import Document

specific_context = """
D'un point de vue idéologique, le droit français désigne un système juridique fondé sur une structure
écrite et codifiée, qui s'oppose tout à la fois aux autres droits européens influencés par le droit
romano-civiliste et aux systèmes de common law.
""" 

document_chain.invoke({
    "input": "C'est quoi le droit français?",
    "context": [Document(page_content=specific_context)]
})

# Sentence Transformers
- Il faut trouver un modèle Sentence Transformers gratuit (donc pas de OpenAI Embeddings ni Ollama Embeddings)
- Ce modèle doit être pre-entrainé pour la comparaison semantique
- Ce modèle doit comprendre le français

Source https://www.sbert.net/docs/pretrained_models.html

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings_model_id = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_id)

/home/adrian/Desktop/NLP/droitGPT/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Corpus
On va se limiter a 3 codes juridiques

## Clean Corpus

In [25]:
import os
import re

data_folder = "data"
relevant_files = ["travail.md", "education.md", "electoral.md"]

clean_data_folder_name = "clean_data"

if not os.path.exists(clean_data_folder_name):
        os.makedirs(clean_data_folder_name)

for file in os.listdir(data_folder):
    if file in relevant_files:
        file_path = data_folder + "/" + file 
        clean_file_path = clean_data_folder_name + "/clean_" + file
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

            # Transform textdata with re here
            pattern1 = re.compile(r'---.*?---', re.DOTALL) # remove title and date 
            text = re.sub(pattern1, '', text)
            
            pattern2 = re.compile(r'#+.*\n') # remove Markdown titles
            text = re.sub(pattern2, '', text)

            pattern2 = re.compile(r'\*\*.*\*\*') # remove Article titles
            text = re.sub(pattern2, '', text)

            pattern3 = re.compile(r'<div.*</div>') # remove html content (simple approach, usually tables)
            text = re.sub(pattern3, '', text)

            pattern4 = re.compile(r'([:;])\n+') # form paragraphs
            text = re.sub(pattern4,lambda x: x.group(1) + ' ', text)

            with open(clean_file_path, "w", encoding="utf-8") as f:
                f.write(text)

In [26]:
import os
import re

from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document


CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0

data_folder = "clean_data"
documents = []
for file in os.listdir(data_folder):
    filename = os.fsdecode(data_folder + "/" + file)
    loader = TextLoader(filename)
    documents += loader.load()

docs = []
for doc in documents:
    texts = re.split('\n+', doc.page_content)
    texts = [text for text in texts if text]
    for text in texts:
        docs.append(Document(page_content = text, metadata=doc.metadata))

In [27]:
print(len(docs))
print()
print(docs[0])
print()
print(docs[100])

35102

page_content="Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et interprofessionnelle fait l'objet d'une concertation préalable avec les organisations syndicales de salariés et d'employeurs représentatives au niveau national et interprofessionnel en vue de l'ouverture éventuelle d'une telle négociation." metadata={'source': 'clean_data/clean_travail.md'}

page_content='Elles sont également applicables au personnel des personnes publiques employé dans les conditions du droit privé.' metadata={'source': 'clean_data/clean_travail.md'}


# Vector Database

In [28]:
from langchain_community.vectorstores import FAISS

vector = FAISS.from_documents(docs, embeddings)

# Querying the Database
On voit que les docs récuperés de la base de données sont en fait similaires au requête

In [15]:
def print_retrieved_docs(docs_and_scores):
    for i, (doc, score) in enumerate(docs_and_scores):
        print(f"Top {i+1}")
        print(doc.page_content)
        print()

In [16]:
query = "L'éducation des jeunes sourds"
docs_and_scores = vector.similarity_search_with_score(query)
print_retrieved_docs(docs_and_scores)

Top 1
L'utilisation de la langue des signes dans l'éducation des jeunes sourds est régie par les articles R. 351-21 à R. 351-26.

Top 2
Les établissements ou services relevant du 2° du I de l'article L. 312-1 du code de l'action sociale et des familles qui soit assurent en leur sein la scolarisation des jeunes sourds, soit contribuent à leur projet personnalisé de scolarisation lorsqu'ils sont scolarisés dans des écoles ou des établissements scolaires, ainsi que les établissements dont la création ou l'extension sont envisagées, élaborent un document annexé au projet d'établissement ou de service relatif aux conditions d'éducation et au parcours scolaire proposés aux jeunes sourds.

Top 3
Dans l'éducation et le parcours scolaire des jeunes sourds, la liberté de choix entre une communication bilingue, langue des signes et langue française, et une communication en langue française est de droit. Un décret en Conseil d'Etat fixe, d'une part, les conditions d'exercice de ce choix pour les j

In [17]:
query = "Contrat de travail à durée déterminée"
docs_and_scores = vector.similarity_search_with_score(query)
print_retrieved_docs(docs_and_scores)

Top 1
III. – Bénéficie également de l'allocation de professionnalisation et de solidarité le travailleur involontairement privé d'emploi qui justifie de 507 heures de travail au cours des dix-huit mois qui précèdent la date de fin de la période d'indemnisation ouverte au titre de la clause de rattrapage prévue aux annexes mentionnées au I.

Top 2
Il peut être conclu afin de pourvoir un emploi permanent qui, par nature, comporte une alternance de périodes travaillées et de périodes non travaillées.

Top 3
La dérogation, prévue à l'article L. 5132-15-1 en cas de cumul avec un autre contrat de travail à temps partiel, à la durée hebdomadaire de travail minimale du titulaire d'un contrat à durée déterminée conclu en application de l'article L. 1242-3, est autorisée par le préfet, après examen par la structure d'insertion par l'activité économique qui emploie le salarié, de la situation de celui-ci au regard de l'emploi et des actions d'accompagnement et de formation conduites dans le cadre

In [19]:
query = "Le surffrage"
docs_and_scores = vector.similarity_search_with_score(query)
print_retrieved_docs(docs_and_scores)

Top 1
de meubles aux enchères publiques

Top 2
de meubles aux enchères publiques</b>

Top 3
Les biens.

Top 4
des courtiers de marchandises assermentés



In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": query})
print(response["answer"])

In [ ]:
response

# TODO
- Clean data before indexing (embeddings computation)
- increase model output tokens ? 